In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/task-1-clickbait-detection-msci-641-s-25/val.jsonl
/kaggle/input/task-1-clickbait-detection-msci-641-s-25/test.jsonl
/kaggle/input/task-1-clickbait-detection-msci-641-s-25/train.jsonl
/kaggle/input/task-1-clickbait-detection-msci-641-s-25/sample_solution.csv


In [2]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    return " ".join(words)

In [4]:
train_path = '/kaggle/input/task-1-clickbait-detection-msci-641-s-25/train.jsonl'
val_path = '/kaggle/input/task-1-clickbait-detection-msci-641-s-25/val.jsonl'
test_path = '/kaggle/input/task-1-clickbait-detection-msci-641-s-25/test.jsonl'

df_train = pd.read_json(train_path, lines=True)
df_val = pd.read_json(val_path, lines=True)
df_test = pd.read_json(test_path, lines=True)

In [5]:
X_train = df_train['postText'].apply(lambda x: clean_text(x[0]))
y_train = df_train['tags'].apply(lambda x: x[0])

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

In [6]:
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000))
])
model.fit(X_train, y_train_enc)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression(max_iter=1000))])

In [7]:
X_val = df_val['postText'].apply(lambda x: clean_text(x[0]))
y_val = df_val['tags'].apply(lambda x: x[0])
y_val_enc = le.transform(y_val)

y_val_pred_enc = model.predict(X_val)
print("Validation Result：")
print(classification_report(y_val_enc, y_val_pred_enc, target_names=le.classes_))

Validation Result：
              precision    recall  f1-score   support

       multi       0.83      0.23      0.36        84
     passage       0.57      0.61      0.59       154
      phrase       0.56      0.73      0.63       162

    accuracy                           0.58       400
   macro avg       0.65      0.52      0.53       400
weighted avg       0.62      0.58      0.56       400



In [8]:
X_test = df_test['postText'].apply(lambda x: clean_text(x[0]))
y_test_pred_enc = model.predict(X_test)
y_test_pred = le.inverse_transform(y_test_pred_enc)

df_submission = pd.DataFrame({
    'id': df_test['id'],
    'spoilerType': y_test_pred
})

df_submission.to_csv('/kaggle/working/prediction_task1.csv', index=False)